In [1]:
# ====================================
# openai tokens
# ====================================

import os
import glob
import codecs
import pickle
import re
import textwrap
from collections import namedtuple

import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, MarkdownTextSplitter
from langchain.vectorstores import FAISS
from pymongo import MongoClient
from langchain.llms import OpenAI

from sys import path

path.append("/opt/configs/ramjet")
import prd


os.environ["OPENAI_API_BASE"] = prd.OPENAI_API
os.environ["OPENAI_API_KEY"] = prd.OPENAI_AZURE_TOKEN


def pretty_print(text: str) -> str:
    text = text.strip()
    return textwrap.fill(text, width=60, subsequent_indent="    ")

In [5]:
prd??

In [2]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
)
from langchain.memory import ConversationBufferWindowMemory


template = """
You are a translation bot. Please translate the content under <<raw>> into English, preserving all formatting and symbols. Only provide the translated text, and do not provide irrelevant instructions, not or explanations. If you encounter code wrapped in markdown or HTML tags, only translate the Chinese characters and maintain the original formatting. If you encounter any content that cannot be translated, please return it unchanged.

<<raw>>
{input}
"""

prompt = PromptTemplate(
    input_variables=["input"], 
    template=template
)
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0, 
    max_tokens=2000) 
conversation = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=False, 
#     memory=ConversationBufferWindowMemory(k=0),
)

In [10]:
# =====================================
# 切块
# =====================================

from datetime import datetime
from pymongo import MongoClient
from langchain.text_splitter import MarkdownTextSplitter, CharacterTextSplitter


DB_HOST = prd.MONGO_HOST
# DB_HOST = "100.97.108.34"  # ubuntu

mongohost = f"mongodb://{prd.MONGO_ADMIN_USER}:{prd.MONGO_ADMIN_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}"
dbconn = MongoClient(host=mongohost)
posts_col = dbconn["blog"]['posts']


# splitter = MarkdownTextSplitter(chunk_size=200,chunk_overlap=0)
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)

def scan_posts_in_mongo():
    cursor = posts_col.find().max_time_ms(0).batch_size(50)
    i = 0
    for docu in cursor: 
        i += 1
        content = docu.get("post_markdown")
        translated_content = ""
        if not content:
            continue
            
        #print(docu["_id"])
        #return
        
        if docu.get("i18n", {}).get("en-us", {}):
            print(f"skip {docu['post_name']}")
            continue
            
        for chunk in content.split("\n"):
            if not chunk.strip():
                translated_content += chunk + "\n"
                continue
            
            if len(chunk) < 400:
                cnt = conversation.predict(input=chunk) + "\n"
            else:
                cnt = ""
                for ichunk in splitter.split_text(chunk): 
                    cnt += conversation.predict(input=ichunk)

            translated_content += cnt
            
        # print(translated_content)

        # save to mongodb
        posts_col.update_one(
            filter={"_id": docu['_id']},
            update={"$set": {"i18n": {
                "updated_at": datetime.now(),
                "en-us": {
                    "post_markdown": translated_content,
                }
            }}}
        )
        
        print(f"{docu.get('post_name')}[{docu.get('_id')}] is ok")
        # return

scan_posts_in_mongo()

skip roc
skip chinesehistory
skip modern-history
skip roc-corner
skip unittest-mock
skip clustering
skip china-parliament
skip docker-centos
skip pyenv
skip gitflow
skip django
skip django-rest
skip bower-gulp
skip literature-1
skip osx-tools-normal
skip jasmine
skip sass-compass
skip angular-learn
skip flask-restful
skip code-style-guide
skip ldap
skip handlebars
skip gou-jian-zhi-fa
skip pygraphviz
skip snmp
skip ansible


InvalidRequestError: Resource not found

In [6]:
conversation.predict(input="你好")

InvalidRequestError: Resource not found